# 1. IMPORTING IMPORTANT LIBRARIES AND THE DATASET

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from prophet import Prophet
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
file_path = "/kaggle/input/global-ai-job-market-and-salary-trends-2025/ai_job_dataset1.csv"
df = pd.read_csv(file_path)

# 2. DATASET CLEANING AND PREPROCESSING

In [ ]:
df_info = df.info()
df_head = df.head()
df_description = df.describe(include='all')

df_info, df_head, df_description

# DATSET OVERVIEW

| Aspect                | Summary                                                              |
| --------------------- | -------------------------------------------------------------------- |
| **Rows**              | 15,000                                                               |
| **Columns**           | 20                                                                   |
| **Key Columns**       | `job_title`, `salary_usd`, `experience_level`, `company_location`    |
| **Salary Range**      | USD salary: min \~16k, max \~410k, median \~107k                     |
| **Experience Range**  | 0 to 19 years                                                        |
| **Job Posting Dates** | Spread across 2024–2025 (for potential time-series analysis)         |
| **Remote Ratio**      | 0 to 100 (perfect for hybrid/remote work trend analysis)             |
| **Skills**            | Listed in text format (`required_skills`) – great for NLP or parsing |
| **Industry Types**    | 15 unique industries                                                 |


In [ ]:
# Convert posting_date and application_deadline to datetime
df['posting_date'] = pd.to_datetime(df['posting_date'], errors='coerce')
df['application_deadline'] = pd.to_datetime(df['application_deadline'], errors='coerce')

# Create a new column to analyze job posting trends over time
df['posting_month'] = df['posting_date'].dt.to_period('M').astype(str)

# Grouping by month to count job postings
monthly_postings = df.groupby('posting_month').size().reset_index(name='job_postings')

# Sort by date for time series visualization
monthly_postings['posting_month'] = pd.to_datetime(monthly_postings['posting_month'])
monthly_postings = monthly_postings.sort_values('posting_month')



# 3. EXPLORATORY DATA ANALYSIS(EDA)

In [ ]:
# Create interactive time series line chart
fig = px.line(monthly_postings,
              x='posting_month',
              y='job_postings',
              title='Monthly AI Job Postings Trend (2024–2025)',
              labels={'posting_month': 'Month', 'job_postings': 'Number of Job Postings'},
              markers=True)

fig.update_layout(xaxis_title='Posting Month',
                  yaxis_title='Number of Job Postings',
                  hovermode='x unified')

fig.show()

# Insight: Monthly AI Job Postings Trend (2024–2025)
* The line chart above visualizes the number of AI job postings across different months based on the posting_date field. Here's a breakdown of what this tells us:

**Observations:**
**Increasing Hiring Trends:**
* There's a noticeable rise in job postings over time, especially in mid to late 2024 and early 2025. This indicates strong market demand and investment in AI talent.

**Seasonality Patterns:**

* Peaks: Certain months exhibit hiring spikes — likely tied to fiscal planning cycles (e.g., January and April).

* Dips: Lower activity around August–September, possibly due to holiday seasons or budget exhaustion in some firms.

**Hiring Surges Before Deadlines:**
* Spikes shortly before major deadlines (fiscal year-end or budget allocations) suggest companies often rush to fill AI positions.

In [ ]:
# Prepare for skill demand analysis
# Split the 'required_skills' column into lists
df['skills_list'] = df['required_skills'].str.split(', ')

# Explode the list to get one skill per row
skills_exploded = df.explode('skills_list')

# Count frequency of each skill
top_skills = skills_exploded['skills_list'].value_counts().reset_index()
top_skills.columns = ['Skill', 'Frequency']

# Display top 20 in a bar chart
plt.figure(figsize=(14, 6))
top_20_skills = top_skills.head(20)
plt.barh(top_20_skills['Skill'][::-1], top_20_skills['Frequency'][::-1])
plt.title("Top 20 Most In-Demand AI Job Skills")
plt.xlabel("Frequency")
plt.ylabel("Skill")
plt.tight_layout()
plt.grid(axis='x')
plt.show()

# Skill Demand Analysis: Top 20 In-Demand AI Job Skills
* The horizontal bar chart above showcases the 20 most frequently required skills across 15,000 AI job postings.

**Observations:**
* **Python** leads the skill demand — unsurprising, given its dominance in AI, data science, and ML.

* **TensorFlow** and **PyTorch** both appear high on the list, indicating strong hiring in deep learning roles.

* **SQL** remains a foundational skill, cutting across roles — from data engineering to analytics.

* **Cloud Platforms (GCP, AWS, Azure)** are prominent, reflecting the shift toward scalable AI solutions in production environments.

* **Tableau** shows high demand — companies value professionals who can also communicate data insights visually.

* **Docker** and **Kubernetes** highlight the importance of MLOps and deployment skills in production AI roles.



In [ ]:
# Analyze top 10 skills by company size
top_skills_list = top_skills['Skill'].head(10).tolist()

# Filter only top skills
filtered_skills = skills_exploded[skills_exploded['skills_list'].isin(top_skills_list)]

# Group by skill and company size, count frequency
skill_company_size = filtered_skills.groupby(['skills_list', 'company_size']).size().reset_index(name='count')

# Pivot for visualization
pivot_size = skill_company_size.pivot(index='skills_list', columns='company_size', values='count').fillna(0)

# Plot as stacked bar chart
pivot_size.plot(kind='bar', stacked=True, figsize=(14, 6))
plt.title("Top 10 Skills by Company Size")
plt.xlabel("Skill")
plt.ylabel("Frequency")
plt.legend(title="Company Size")
plt.tight_layout()
plt.grid(axis='y')
plt.show()

# Skill Demand by Company Size
* The stacked bar chart above shows how demand for the Top 10 AI skills varies across companies of different sizes:

**Observations:**
**Large Companies ('L') Dominate in Hiring for Core AI Skills:**

* **Python**, **TensorFlow**, and **SQL** show much higher demand from large firms.

* This suggests mature AI teams that require a broad stack of skills, from programming to cloud and deployment.

**Medium-Sized Firms ('M') Favor Balanced Skillsets:**

* They demand a mix of core tech (**Python**, **SQL**) and data visualization tools like **Tableau**, possibly for client-facing roles or internal reporting.

**Small Companies ('S') Prefer Lightweight, Versatile Tools:**

* Skills like **Tableau**, **Docker**, and **PyTorch** show stronger relative demand from smaller firms, likely due to need for multi-skilled candidates in lean teams.

In [ ]:
# Group by skill and industry for top 10 skills
skill_industry = filtered_skills.groupby(['skills_list', 'industry']).size().reset_index(name='count')

# Pivot for heatmap visualization
pivot_industry = skill_industry.pivot(index='skills_list', columns='industry', values='count').fillna(0)

# Plot heatmap using seaborn
import seaborn as sns
plt.figure(figsize=(14, 6))
sns.heatmap(pivot_industry, annot=True, fmt='.0f', cmap='YlGnBu', cbar_kws={'label': 'Frequency'})
plt.title("Top 10 Skills by Industry")
plt.xlabel("Industry")
plt.ylabel("Skill")
plt.tight_layout()
plt.show()

# Skill Demand by Industry (Heatmap)
* The heatmap above visualizes how the Top 10 AI job skills vary in demand across different industries.

**Key Insights:**
* Python is universally dominant — appearing at the top in every industry, from **Automotive** to **Finance** to **Government**.**It's the backbone of modern AI development.**

**TensorFlow** and **PyTorch** show strong concentration in **Technology**, **Healthcare**, and **Automotive**:

* These industries are actively adopting deep learning for applications like **self-driving**, **diagnostics**, and **personalization**.

**SQL** and **Tableau** are highly demanded in **Finance**, **Government**, and **Retail**:

* These sectors rely on structured data, reporting, and decision intelligence.

**Cloud Platforms (GCP, AWS):**

* More prevalent in **Technology** and **Consulting**, where cloud-based AI deployment and scalability are vital.

**Docker** and **Kubernetes** are slightly more emphasized in **Consulting**, **Tech**, and **Defense**:

* This reflects a strong need for MLOps and infrastructure knowledge in complex deployment environments.

In [ ]:
# Set plot style
sns.set(style="whitegrid")

# Prepare salary by role
salary_by_role = df.groupby('job_title')['salary_usd'].median().sort_values(ascending=False).head(15).reset_index()

# Plot top 15 job titles by median salary
plt.figure(figsize=(14, 6))
sns.barplot(data=salary_by_role, x='salary_usd', y='job_title', palette='viridis')
plt.title("Top 15 AI Job Titles by Median Salary (USD)")
plt.xlabel("Median Salary (USD)")
plt.ylabel("Job Title")
plt.tight_layout()
plt.show()

# Top 15 AI Job Titles by Median Salary (USD)
* This bar chart highlights the AI roles offering the highest median salaries across the dataset.

**Insights:**
**Executive & Head-Level Roles dominate the top**:

* **Head of AI**, **AI Architect**, and **Chief Data Scientist** lead the salary charts, reflecting their strategic importance.

**Specialist roles in Computer Vision, ML Ops, and Robotics also command high salaries:**

* These fields are often complex, niche, and demand hands-on innovation.

* Popular roles like **Data Scientist** and **Data Engineer**, while common, still fetch competitive salaries due to high demand and foundational importance.

* **NLP Engineer** and **AI Research Scientist** also appear — suggesting research-heavy positions are well-compensated.|

In [ ]:
# Prepare salary by experience level
salary_by_experience = df.groupby('experience_level')['salary_usd'].median().reset_index()

# Map experience codes to meaningful labels
experience_map = {
    'EN': 'Entry-level',
    'MI': 'Mid-level',
    'SE': 'Senior-level',
    'EX': 'Executive-level'
}
salary_by_experience['experience_level'] = salary_by_experience['experience_level'].map(experience_map)

# Sort by hierarchy
experience_order = ['Entry-level', 'Mid-level', 'Senior-level', 'Executive-level']
salary_by_experience['experience_level'] = pd.Categorical(salary_by_experience['experience_level'], categories=experience_order, ordered=True)
salary_by_experience = salary_by_experience.sort_values('experience_level')

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=salary_by_experience, x='experience_level', y='salary_usd', palette='magma')
plt.title("Median Salary by Experience Level")
plt.xlabel("Experience Level")
plt.ylabel("Median Salary (USD)")
plt.tight_layout()
plt.show()

# Median Salary by Experience Level
* This chart clearly illustrates how AI job salaries scale with experience, using median values for each category.

**Key Insights:**
* **Executive-Level Roles (EX) stand out dramatically, with median salaries nearly double that of senior-level roles.**
* These roles often involve leadership, strategy, and oversight responsibilities.

* **Senior-Level Roles (SE) provide a significant jump from mid-level, reflecting deep expertise, team leadership, or specialized domain skills.**

* **Entry-Level vs Mid-Level:**

* There’s a meaningful increase from Entry (EN) to Mid (MI), emphasizing the importance of 2–5 years of experience in commanding better pay.

In [ ]:
# Group by company location and get median salary
top_locations = df.groupby('company_location')['salary_usd'].median().sort_values(ascending=False).head(15).reset_index()

# Plot top 15 paying countries
plt.figure(figsize=(14, 6))
sns.barplot(data=top_locations, x='salary_usd', y='company_location', palette='coolwarm')
plt.title("Top 15 Company Locations by Median Salary (USD)")
plt.xlabel("Median Salary (USD)")
plt.ylabel("Company Location")
plt.tight_layout()
plt.show()

# Top 15 Company Locations by Median Salary (USD)
* This bar chart ranks the top-paying countries based on the location of the employing company.

**Insights:**
* **Switzerland**, **USA**, and **Japan** top the list — these are high-income economies with strong AI ecosystems and global R&D hubs.

* **Switzerland** consistently ranks high due to currency strength, cost of living, and strong innovation policies.

* Nordic Countries (**Sweden**, **Finland**) and **Germany** show strong median salaries, likely due to:

          * Mature tech industries

          * Emphasis on high-skill employment

          * Government-backed AI investments

* **Singapore** and **Australia** also make the top 10, reflecting their booming AI startup and innovation scenes.

* **China** and **India** are absent from the top 15 here, possibly due to:

         * Broader salary range and volume of jobs at the lower end

         * Local currency to USD conversion



In [ ]:
# Group by employee residence and get median salary
top_residences = df.groupby('employee_residence')['salary_usd'].median().sort_values(ascending=False).head(15).reset_index()

# Plot top 15 residences by salary
plt.figure(figsize=(14, 6))
sns.barplot(data=top_residences, x='salary_usd', y='employee_residence', palette='Spectral')
plt.title("Top 15 Employee Residences by Median Salary (USD)")
plt.xlabel("Median Salary (USD)")
plt.ylabel("Employee Residence")
plt.tight_layout()
plt.show()

# Top 15 Employee Residences by Median Salary (USD)
* This chart showcases where employees live — regardless of where the company is — and how that correlates with their salary in USD.

**Insights:**
* Employees in Switzerland, USA, Singapore, and Germany earn the most — mirroring trends seen in company location.

* These countries likely offer both on-site and remote roles with top-tier compensation.

**Remote Advantage?**

* If a candidate lives in a high-paying country and works remotely, they can still retain premium wages.

* Conversely, lower-residence countries may offer lower salary baselines even when working for global companies.

**Disparity Example:**

* Even if the same job is offered remotely, employee residence affects the pay structure — due to local laws, living costs, and taxation systems.

**Talent Migration Indicator:**

* High-income countries being dominant in both company location and residence suggests concentration of top AI talent and opportunities in these geographies.

In [ ]:
# Group by remote ratio buckets (0, 50, 100) and compute median salary
remote_salary = df.groupby('remote_ratio')['salary_usd'].median().reset_index()

# Plot salary vs remote ratio
plt.figure(figsize=(10, 5))
sns.barplot(data=remote_salary, x='remote_ratio', y='salary_usd', palette='Blues')
plt.title("Median Salary by Remote Work Ratio")
plt.xlabel("Remote Work Ratio (%)")
plt.ylabel("Median Salary (USD)")
plt.tight_layout()
plt.show()

# Median Salary by Remote Work Ratio
* This bar chart breaks down median salaries by remote work percentage, helping us evaluate if working from home affects pay.

**Key Takeaways:**

 **50% Remote Roles offer the highest median salary:**

* These are often hybrid roles combining flexibility with strategic in-office responsibilities.

* They may also reflect mid to senior positions, which companies are willing to pay more for while retaining partial office presence.

**Fully Remote (100%) Roles:**

* Surprisingly, they earn less than hybrid roles, possibly due to:

* Cost optimization by hiring from low-income geographies

* Remote-only jobs being more execution-driven than strategic

**On-site (0% Remote) Roles**:

* These are the least compensated on average.

* They often exist in locations with lower living costs or manual process dependencies.

In [ ]:
# Prepare remote ratio by experience level
remote_exp = df.groupby('experience_level')['remote_ratio'].mean().reset_index()
remote_exp['experience_level'] = remote_exp['experience_level'].map(experience_map)
remote_exp['experience_level'] = pd.Categorical(remote_exp['experience_level'],
                                                categories=experience_order,
                                                ordered=True)
remote_exp = remote_exp.sort_values('experience_level')

# Plot: Average remote ratio by experience level
plt.figure(figsize=(10, 5))
sns.barplot(data=remote_exp, x='experience_level', y='remote_ratio', palette='cividis')
plt.title("Average Remote Work Ratio by Experience Level")
plt.xlabel("Experience Level")
plt.ylabel("Average Remote Ratio (%)")
plt.tight_layout()
plt.show()

#  Average Remote Work Ratio by Experience Level
* This chart reveals how remote work opportunities vary by experience tier — measured by the average percentage of remote work assigned to each level.

**Insights:**

**Mid-Level (MI) and Entry-Level (EN) roles have the highest average remote ratios:**

* Companies may be more open to remote arrangements for junior talent in supportive or individual contributor roles.

* This also allows for cost-efficient hiring from remote geographies.

**Executive-Level (EX) roles have the lowest remote ratio:**

* Senior executives are more likely to be involved in strategic planning, leadership meetings, and decision-making, which often require in-person presence.

* These roles also involve sensitive operations or cross-functional coordination, making physical proximity valuable.

* Senior-Level (SE) professionals sit in the middle — possibly alternating between remote flexibility and leadership responsibilities.

In [ ]:
# Calculate average remote ratio by industry
remote_industry = df.groupby('industry')['remote_ratio'].mean().sort_values(ascending=False).reset_index()

# Plot top industries by remote work ratio
plt.figure(figsize=(14, 6))
sns.barplot(data=remote_industry, x='remote_ratio', y='industry', palette='coolwarm')
plt.title("🏭 Average Remote Work Ratio by Industry")
plt.xlabel("Average Remote Ratio (%)")
plt.ylabel("Industry")
plt.tight_layout()
plt.show()

# Average Remote Work Ratio by Industry
* This chart ranks industries by how much they embrace remote work, using the average remote work ratio across all job postings.

**Key Observations:**

**Technology & Consulting Firms lead with the highest remote flexibility:**

* These industries inherently support remote operations, given their digital infrastructure and cloud-native work culture.

**Education and Finance also rank high:**

* EdTech and digital finance platforms allow seamless remote delivery.

* Finance often involves back-end roles that can function effectively without on-site presence.

**Healthcare & Defense show lower remote ratios:**

* These sectors often involve regulatory, data privacy, or on-site physical constraints, limiting full remote setups.

**Automotive and Retail are on the lower end:**

* AI roles in these industries might be embedded in hardware development, logistics, or manufacturing systems, requiring closer integration with physical assets.

In [ ]:
# Group by company size and calculate median salary and average benefits score
company_benefits_salary = df.groupby('company_size').agg({
    'salary_usd': 'median',
    'benefits_score': 'mean'
}).reset_index()

# Sort by company size (small, medium, large)
company_size_order = ['S', 'M', 'L']
company_benefits_salary['company_size'] = pd.Categorical(company_benefits_salary['company_size'],
                                                         categories=company_size_order,
                                                         ordered=True)
company_benefits_salary = company_benefits_salary.sort_values('company_size')

# Plotting side-by-side bar plots for salary and benefits
fig, ax1 = plt.subplots(figsize=(10, 5))

sns.barplot(data=company_benefits_salary, x='company_size', y='salary_usd', ax=ax1, color='skyblue', label='Median Salary (USD)')
ax1.set_ylabel("Median Salary (USD)", color='skyblue')
ax1.set_xlabel("Company Size")
ax1.set_title("Company Size vs Median Salary & Average Benefits Score")

# Create a second y-axis
ax2 = ax1.twinx()
sns.barplot(data=company_benefits_salary, x='company_size', y='benefits_score', ax=ax2, color='lightcoral', label='Average Benefits Score')
ax2.set_ylabel("Average Benefits Score", color='lightcoral')

plt.tight_layout()
plt.show()

# Company Size vs Median Salary & Benefits Score

**This dual-axis chart compares how company size affects:**

* Median salary in USD

* Average benefits score (scale: 5 to 10)

**Key Insights:**

**Large Companies (L) offer the highest salaries and strongest benefits:**

* Reflects their ability to attract and retain top-tier talent with comprehensive compensation packages.

* Often have structured HR systems, stock options, insurance plans, and wellness benefits.

**Medium-Sized Firms (M) strike a balance:**

* Salaries are competitive, and benefits are decent.

* May be growth-phase firms with funding but fewer resources than tech giants.

**Small Companies (S) lag behind in both metrics:**

* Lower salary budgets and fewer formalized benefits.

* Often rely on equity offers, startup culture perks, and flexible work environments to attract talent.

# 4. MODEL TRAINING

 # TIME SERIES ANALYSIS USING PROPHET 

In [ ]:
from prophet import Prophet

# Prepare data for Prophet model
# Prophet requires columns 'ds' (datetime) and 'y' (value to forecast)
prophet_df = monthly_postings.rename(columns={"posting_month": "ds", "job_postings": "y"})

# Fit the Prophet model
prophet_model = Prophet()
prophet_model.fit(prophet_df)

# Forecast next 6 months
future_plotly = prophet_model.make_future_dataframe(periods=6, freq='M')
forecast_plotly = prophet_model.predict(future_plotly)

# Create Plotly figure
fig = go.Figure()

# Actual data points
fig.add_trace(go.Scatter(
    x=prophet_df['ds'], y=prophet_df['y'],
    mode='markers', name='Actual Job Postings'
))

# Forecasted trend
fig.add_trace(go.Scatter(
    x=forecast_plotly['ds'], y=forecast_plotly['yhat'],
    mode='lines', name='Forecasted Trend'
))

# Upper bound
fig.add_trace(go.Scatter(
    x=forecast_plotly['ds'], y=forecast_plotly['yhat_upper'],
    mode='lines', name='Upper Bound',
    line=dict(width=0), showlegend=False
))

# Lower bound + fill
fig.add_trace(go.Scatter(
    x=forecast_plotly['ds'], y=forecast_plotly['yhat_lower'],
    mode='lines', name='Confidence Interval',
    line=dict(width=0), fill='tonexty',
    fillcolor='rgba(173,216,230,0.4)', showlegend=True
))

# Layout settings
fig.update_layout(
    title="Forecast of AI Job Postings (Next 6 Months)",
    xaxis_title="Month",
    yaxis_title="Expected Job Postings",
    hovermode='x unified'
)

fig.show()

# Key Insights from Prophet Forecast Analysis:

**Slight Downward Trend in AI Job Postings**

* The red trend line indicates a subtle decline in the number of AI job postings moving forward.

* This doesn't suggest a collapse, but rather a market cooling after a peak.

**It could reflect:**

    * Budget tightening

    * Post-hiring surge correction

    * Shift from hiring to AI tool adoption by non-tech industries

**Short-Term Volatility but Long-Term Stability**

* **The wide blue band (confidence interval) suggests:**

      * While the median forecast shows mild decline, actual postings could still fluctuate significantly month to month (from ~870 to ~1000).

      * Indicates uncertainty in market behavior — possibly due to economic, regulatory, or tech disruption factors.

**Cyclicality Not Strongly Evident**

* The absence of sharp up/down movement implies low seasonality.

* Job postings may not follow clear seasonal hiring cycles — instead, trends are more trend-driven (e.g., funding cycles, product launches, global events).

**Forecasted Job Range (Next 6 Months)**

* Lower Bound: ~860–870 job postings/month

* Upper Bound: ~1000–1010 job postings/month

* Central Estimate: around 930–940

* This range is wide enough to be cautious, but centered closely enough to show stability in hiring.

# SALARY PREDICTION USING RANDOMFORESTREGRESSOR

In [ ]:
# Select features and target
features = [
    'experience_level', 'employment_type', 'company_location',
    'employee_residence', 'remote_ratio', 'company_size',
    'education_required', 'industry', 'job_description_length', 'benefits_score'
]
target = 'salary_usd'

# Subset the dataset
X = df[features]
y = df[target]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define categorical and numerical columns
categorical_features = [
    'experience_level', 'employment_type', 'company_location',
    'employee_residence', 'company_size', 'education_required', 'industry'
]
numerical_features = ['remote_ratio', 'job_description_length', 'benefits_score']

# Preprocessing for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a full pipeline with preprocessing and model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, r2

# Salary Prediction Model (Random Forest) – Results

**After optimizing for execution time, the model shows the following performance:**

**Metric	Value**
* Mean Absolute Error:	$15,796
* R² Score (Accuracy):	0.88 (≈88%)

**Interpretation:**

* The model can predict AI job salaries with ~88% accuracy using only job attributes like:

      * Experience level

      * Location

      * Remote ratio

      * Education and company size

      * Industry and benefits score

* An average prediction is off by around ~15800 USD,  which is reasonable given the salary range spans 16K USD to 410K USD.